# Iterators and Generators

## Iterators

An `iterator` is nothing more than a container object what implements the iterator protocol. It's based on two method:

* `__next__`, which returns the next item of container
* `__iter__`, which returns the iterator itself

Iterators can be created with a sequence using the `iter` built-in function.

In [ ]:
i = iter('abc')
next(i)
next(i)
next(i)
next(i)

In [ ]:
class MyIterator(object):
    def __init__(self, step):   
        self.step = step
    def __next__(self):
        """Returns the next element."""
        if self.step == 0:
            raise StopIteration
        self.step -= 1
        return self.step
    def __iter__(self):
        """Returns the iterator itself."""
        return self
        
for el in MyIterator(4):
    print(el)

## Generators

In [ ]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a + b

fib = fibonacci()

[next(fib) for i in range(10)]

`generator` can interact with the code called with the `next` method. `yield` becomes an expression, and a value can be passed along with a new method called `send`.

In [ ]:
def psychologist():
    print('Please tell me your problems')
    while True:
        answer = (yield)
        if answer is not None:
            if answer.endswith('?'):
                print ("Don't ask yourself too much questions")
            elif 'good' in answer:
                print("A that's good, go on")
            elif 'bad' in answer:
                print("Don't be so negative")

free = psychologist()
next(free)
free.send('I feel bad')
free.send("Why I shouldn't ?")
free.send("ok then i should find what is good for me")

## Coroutines

A `coroutine` is a function that can be suspended and resumed, and can have multiple entry points.

## Generator Expresstions

In [ ]:
iter = (x**2 for x in range(10) if x % 2 == 0)

for x in iter:
    print(x)

## The itertools module

### islice: the window iterator

In [ ]:
import itertools
import numpy as np

numbers = np.random.randint(5, size=(10, 10))

for n1 in numbers:
    for n2 in itertools.islice(n1, 4, None):
        print(n2)

One can use `islice` every time to extract data located in a particular position in a stream. This can be a file in a special format using records for instance, or a stream that presents data encapsulated with metadata, like a **SOAP** envelope, for example. In that case, `islice` can be seen as a window that slides over each line of data.

### tee: the back and forth iterator

In [ ]:
import itertools

def with_head(iterable, headsize=1):
    a, b = itertools.tee(iterable)
    return list(itertools.islice(a, headsize)), b

seq = range(1, 10)
with_head(seq)
with_head(seq, 4)

### groupby: the uniq iterator

The `groupby` function works a little like the Unix command `uniq`. It is able to group the duplicate elements from an iterator, as long as they are adjacent. A function can be given to the function for it to compare the elements. Otherwise, the identity comparison is used.

In [ ]:
from itertools import groupby

def compress(data):
    return ((len(list(group)), name)
           for name, group in groupby(data))

def decompress(data):
    return (car * size for size, car in data)

compressed = compress('get uuuuuuuuuuuuuuuuuup')
''.join(decompress(compressed))

### Chaining iterables together

In [ ]:
import itertools

list(itertools.chain(range(3), range(4), range(5)))

## Zipping iterables together

In [ ]:
list(zip(range(3), reversed(range(5))))

The `zip` function is particularly useful in creating dictionaries, because one sequence can be used to supply the keys, while another supplies the values. Using **zip()** can join these together into the proper parings, which can then be passed directly into a new **dict()**.

In [ ]:
keys = map(chr, range(97, 102))
values = range(1, 6)
dict(zip(keys, values))

# Collections

## Sets

* add
* update
* remove
* discard
* pop
* clear
* union
* intersection
* difference
* symmetric_difference
* issubset
* issuperset

## Named tuples

In [ ]:
from collections import namedtuple

Point = namedtuple('Point', 'x y')
point = Point(13, 25)
(point.x, point.y)

## Ordered dictionaries

In [ ]:
from collections import OrderedDict
d = OrderedDict((value, str(value)) for value in range (10) if value > 5)
d[10] = '10'

d

## Dictionaries and defaults

In [ ]:
from collections import defaultdict

def count_words(text):
    count = defaultdict(int)
    for word in text.split(' '):
        count[word] += 1
    return count

# Decorators

## How to write a decorator

A generic pattern is:

In [ ]:
def mydecorator(function):
    def _mydecorator(*args, **kw):
        # do some stuff before the real function gets called
        res = function(*args, **kw)
        # do some stuff after
        return res
    
    return _mydecorator

In [ ]:
When arguments are needed for the decorator to work on, a second level of wrapping has to be used.

In [ ]:
def mydecorator(arg1, arg2):
    def _mydecorator(function):
        def __mydecorator(*args, **kw):
            # do some stuff before the real function gets called
            res = function(*args, **kw)
            # do some stuff after
            return res
        
        # returns the sub-function
        return __mydecorator
    return _mydecorator

## Argument checking

In [ ]:
rpc_info = {}

def xmlrpc(in_=(), out=(type(None),)):
    def _xmlrpc(function):
        # registering the signature
        func_name = function.__name__
        rpc_info[func_name] = (in_, out)

        def _check_types(elements, types):
            """Subfunction that checks the types."""
            if len(elements) != len(types):
                raise TypeError('argument count is wrong')
            typed = enumerate(zip(elements, types))
            for index, couple in typed:
                arg, of_the_right_type = couple
                if isinstance(arg, of_the_right_type):
                    continue
                raise TypeError('arg #%d should be %s' % \
                    (index, of_the_right_type))

        # wrapped function
        def __xmlrpc(*args): # no keywords allowed
            # checking what goes in
            checkable_args = args[1:] # removing self
            _check_types(checkable_args, in_)
            # running the function
            res = function(*args)

            # checking what goes out
            if not type(res) in (tuple, list):
                checkable_res = (res,)
            else:
                checkable_res = res
            _check_types(checkable_res, out)

            # the function and the type
            # checking succeeded
            return res
        return __xmlrpc
    return _xmlrpc

In [ ]:
class RPCView(object):
    @xmlrpc((int, int))   # two int -> None
    def meth1(self, int1, int2):
        print('received %d and %d' % (int1, int2))
    
    @xmlrpc((str,), (int,))   # string -> int
    def meth2(self, phrase):
        print('received %s' % phrase)

rpc_info

In [ ]:
my = RPCView()

my.meth1(1, 2)
my.meth2(2)

## Caching

In [ ]:
import time
import hashlib
import pickle
from itertools import chain

cache = {}
def is_obsolete(entry, duration):
    return time.time() - entry['time'] > duration

def compute_key(function, args, kw):
    key = pickle.dumps((function.__name__, args, kw))
    return hashlib.sha1(key).hexdigest()

def memoize(duration=10):
    def _memoize(function):
        def __memoize(*args, **kw):
            key = compute_key(function, args, kw)
            
            if (key in cache and 
                not is_obsolete(cache[key], duration)):
                print('we got a winner')
                return cache[key]['value']

            result = function(*args, **kw)
            cache[key] = {'value': result, 'time': time.time()}
            
            return result
        return __memoize
    return _memoize

In [ ]:
@memoize()
def very_very_very_complex_stuff(a, b):
    return a + b

very_very_very_complex_stuff(2, 2)
very_very_very_complex_stuff(2, 2)

cache

## Proxy

In [ ]:
class User(object):
    def __init__(self, roles):
        self.roles = roles
        
class Unauthorized(Exception):
    pass

def protect(role):
    def _protect(function):
        def __protect(self, *args, **kw):
            user = self.user
            if user is None or role not in user.roles:
                raise Unauthorized("I won't tell you")
            return function(self, *args, **kw)
        return __protect
    return _protect

class MySecrets(object):
    def __init__(self, user):
        self.user = user
        
    @protect('admin')
    def waffle_recipe(self):
        print('use tons of butter!')

tarek = User(('admin', 'user'))
bill = User(('user',))        
        
these_are = MySecrets(tarek)
#user = tarek
these_are.waffle_recipe()

these_are = MySecrets(bill)
these_are.waffle_recipe()

## Context provider

A context decorator makes sure that the function can run in the correct context, or run some code before and after the function. In other words, it sets and unsets a specific execution environment. For example, when a data item has to be shared among several threads, a lock has to be used to ensure that it is protected from multiple access.

In [ ]:
from threading import RLock

lock = RLock()
def synchronized(function):
    def _synchronized(*args, **kw):
        lock.acquire()
        try:
            return function(*args, **kw)
        finally:
            lock.release()
    return _synchronized

@synchronized
def thread_safe():   # make sure it locks the resource
    pass

# with and contextlib

In [ ]:
class Context(object):
    def __enter__(self):
        print('entering the zone')
    def __exit__(self, exception_type, exception_value, exception_traceback):
        print('leaving the zone')
        if exception_type is None:
            print('with no error')
        else:
            print('with an error {}'.format(exception_value))
            
with Context():
    print('I am the zone')
    
with Context():
    print('I am the buggy zone')
    raise TypeError('I am the the bug')

## The contextlib module

In [ ]:
from contextlib import contextmanager

@contextmanager
def context():
    print('entering the zone')
    try:
        yield
    except Exception as e:
        print('with an error {}'.format(e))
        raise e # we need to  re-raise here
    else:
        print('with no error')

The two other helps provided by this module are:

* `closing(element)`: This is a `contextmanager` decorated function that yields an element, and then calls the element's close method on exit. This is useful for classes that deal with streams, for instance.
* `nexted(context1, context2, ...)`: This is a function that will combine contexts and make nested `with` calls with them.

## Context example

In [ ]:
import logging
from contextlib import contextmanager

@contextmanager
def logged(klass, logger):
    def _log(f):
        def __log(*args, **kw):
            logger(f, args, kw)
            return f(*args, **kw)
        return __log

    for attribute in dir(klass):
        if attribute.startswith('_'):
            continue
        element = getattr(klass, attribute)
        setattr(klass, '__logged_ {}'.format(attribute), element)
        setattr(klass, attribute, _log(element))

    yield

    for attribute in dir(klass):
        if not attribute.startswith('__logged_'):
            continue

        element = getattr(klass, attribute)
        setattr(klass, attribute[len('__logged_'):], element)
        delattr(klass, attribute)

class One(object):
    def _private(self):
        pass

    def one(self, other):
        self.two()
        other.thing(self)
        self._private()

    def two(self):
        pass

class Two(object):
    def thing(self, other):
        other.two()

calls = []
def called(meth, args, kw):
    calls.append(meth.__name__)

with logged(One, called):
    one = One()
    two = Two()
    one.one(two)

calls

# Subclassing built-in types

In [ ]:
class DistinctError(Exception):
    pass

class distinctdict(dict):
    def __setitem__(self, key, value):
        try:
            existing_key = [k for (k, v) in self.items() if v == value and k != key]
            
            if existing_key:
                raise DistinctError("This value already exists for {}"
                                    .format(str(existing_key[0])))
        except ValueError:
            pass
        
        super(distinctdict, self).__setitem__(key, value)

my = distinctdict()
my['key1'] = 'value'
my['key2'] = 'value'

# Accessing methods from superclasses

In [ ]:
class Mama(object):
    def says(self):
        print('Mama says')

class Sister(Mama):
    def says(self):
        super(Sister, self).says()
        print('Sister says')

anita = Sister()
anita.says()

## Best practices

* Multiple inheritance should be avoided.
* `super` usage has to be consistent. In a class hierarchy, `super` should be used everywhere or nowhere. Mixing `super` and classic calls is a confusing practice.
* Don't mix old-style and new-style classes. Having a code base with both results in a varying MRO behavior.
* Class hierarchy has to be looked over when a parent class is called. To avoid any problems, every time a parent class is called, a quick glance at the involved MRP (with `__mro__`) has to be done.

# Exception

## Exception example

In [ ]:
import logging

def count_lines(filename):
    """
    Count the number of lines in a file. If the file can't be opened,
    it should be treated the same as if it was empty.
    """
    file = None
    try:
        file = open(filename, 'r')
        lines = file.readlines()
    except TypeError as e:
        # the filename wasn't valid for use with the filesystem.
        logging.error(e)
        return 0
    except EnvironmentError as e:
        # Something went wrong reading the file
        logging.error(e)
        return 0
    except UnicodeDecodeError as e:
        # The contents of the file were in a unknown encoding
        logging.error(e)
        return 0
    else:
        return len(lines)
    finally:
        if file:
            file.close()

# Emulating the python 3.x print function

In [ ]:
import sys

def print3(*args, **kargs):
    sep = kargs.get('sep', ' ')  # Keyword arg defaults
    end = kargs.get('end', '\n')
    file = kargs.get('file', sys.stdout)
    output = ''
    first = True
    for arg in args:
        output += ('' if first else sep) + str(args)
        first = False
    flie.write(output + end)
    

# Importing code

## Fallback imports

In [ ]:
try:
    # Use the new library if available. Added in Python 2.5
    from hashlib import md5
except ImportError:
    # Compatible functionality provided prior to Python 2.5
    from md5 import new as md5

In [ ]:
try:
    import docutils
except ImportError:
    docutils = None
    
if docutils:
    pass

## Importing from the Future

If you try to import a feature from **__future** that already exists in the version of Python you're using, it doesn't do anything. The feature is already available, so no changes have to be made, but it also doesn't raise any exceptions.

## Relative imports

Python allows you to specify a relative path to the module you'd like to import, so you can move around an entire package, if necessary, with a minimal modifications required. The preferred syntax for this is to specify part of the module's path with one or more periods, indicating how far up the path to look for the module. For example, if the **ace.shopping.cart** module needs to import from **acme.billing**, the two following import patterns are identical.

```python
from acme import billing
from .. import billing
```

A single period allows you to import from the current package, so **ace.shopping.gallery** could be imported as  **from . import gallery**. Alternatively, if you're looking to just import something from that module, you could instead simply prefix the module path with the necessary periods, then specify the names to import as usual: **from .gallery import Image**.

## Using keyword-only arguments

In [ ]:
" Use 3.x only keyword-only args"
import sys

def print3(*args, sep='', end='\n', file=sys.stdout):
    output = ''
    first = True
    for arg in args:
        output += ('' if first else sep) + str(args)
        first = False
    flie.write(output + end)

In [ ]:
" Use 2.x/3.x keyword args deletion with defaults"
import sys

def print3(*args, **kargs):
    sep = kargs.pop('sep', ' ')
    end = kargs.pop('end', '\n')
    file = kargs.pop('file', sys.stdout)
    if kargs: raise TypeError('extra keywords: %s' % kargs)
    output = ''
    first = True
    for arg in args:
        output += ('' if first else sep) + str(args)
        first = False
    flie.write(output + end)

## The __import__() function

In [ ]:
os = __import__('os.path')
path = os.path

In [ ]:
def import_child(module_name):
    module = __import__(module_name)
    for layer in module_name.split('.')[1:]:
        module = getattr(module, layer)
    return module

path = import_child('os.path')

In [ ]:
An alternate approach takes advantage of Python's own module caching mechanism to take the extra processing out the picture.

In [ ]:
import sys

def import_child(module_name):
    module = __import__(module_name)
    return sys.modules[module_name] 

path = import_child('os.path')

## The importlib module

In [ ]:
from importlib import import_module

path = import_module('os.path')

**import_module** takes relative imports into account by also accepting a **package** attribute that defines the reference point from which the relative path should be resolved. This is easily done when calling the function, simply by passing in the always-global **__name__** variable, which holds the module path that was used to import the current module in the first place.

In [ ]:
import_module('.utils', package=__name__)

## Transitive module reloads

In [ ]:
import types
from imp import reload

def status(module):
    print('reloading ' + module.__name__)

def tryreload(module):
    try:
        reload(module)
    except:
        print('Failed: %s' % module)

def transitive_reload(module, visited):
    if not module in visited:
        status(module)
        tryreload(module)
        visited[module] = True
        for attrobj in module.__dict__.values():
            if type(attrobj) == types.ModuleType:
                transitive_reload(attrobj, visited)

def reload_all(*args):
    visited = {}
    for arg in args:
        if type(arg) == types.ModuleType:
            transitive_reload(attrobj, visited)

def tester(reloader, modname):
    import importlib, sys
    if len(sys.argv) > 1: modname = sys.argv[1]
    module = importlib.import_module(modname)
    reloader(module)

if __name__ == '__main__':
    tester(reload_all, 'reloadall')

### Alternative implementation1

In [ ]:
import types
from imp import reload
from reloadall import status, tryreload, tester

def transitive_reload(objects, visited):
    for obj in objects:
        if type(obj) == types.ModuleType and obj not in visited:
            status(obj)
            tryreload(obj)
            visited.add(obj)
            transitive_reload(obj.__dict__.value(), visited)

def reload_all(*args):
    transitive_reload(args, set())

if __name__ == '__main__':
    tester(reload_all, 'reloadall2')

### Alternative implementation2

In [ ]:
import types
from imp import reload
from reloadall import status, tryreload, tester

def transitive_reload(modules, visited):
    while modules:
        next = modules.pop()
        status(next)
        tryreload(next)
        visited.add(next)
        modules.extend(x for x in next.__dict__.values()
            if type(x) == types.ModuleType and x not in visited)

def reload_all(*modules):
    transitive_reload(list(modules), set())

if __name__ == '__main__':
    tester(reload_all, 'reloadall3')

# Operator overloading

## Common operator overloading methods

| Method                                                                            | Impements                    | Called for                                                                                                                              |
| --------                                                                          | ---------------------------- | ------------------------------------                                                                                                    |
| \_\_init\_\_                                                                      | Constructor                  | Object creation: X = Class(args)                                                                                                        |
| \_\_del\_\_                                                                       | Destructor                   | Object reclamation of X                                                                                                                 |
| \_\_add\_\_                                                                       | Operator +                   | X + Y, X+= Y if no \_\_iadd\_\_                                                                                                         |
| \_\_or\_\_                                                                        | Operator &#124; (bitwise OR) | X &#124; Y, X&#124;= Y if no \_\_ior\_\_                                                                                                |
| \_\_repr\_\_, \_\_str\_\_                                                         | Printing, conversions        | print(X), repr(X), str(X)                                                                                                               |
| \_\_call\_\_                                                                      | Function calls               | X(\*args, \*\*kargs)                                                                                                                    |
| \_\_getattr\_\_                                                                   | Attribute fetch              | X.undefined                                                                                                                             |
| \_\_setattr\_\_                                                                   | Attribute assignment         | X.any = value                                                                                                                           |
| \_\_delattr\_\_                                                                   | Attribute deletion           | del X.any                                                                                                                               |
| \_\_getattribute\_\_                                                              | Attribute fetch              | X.Any                                                                                                                                   |
| \_\_getitem\_\_                                                                   | Indexing, slicing, iteration | X[key], X[i:j], for loops and other iterations if no  \_\_iter\_\_                                                                      |
| \_\_setitem\_\_                                                                   | Index and slice assignment   | X[key] = value, X[i:j] = iterable                                                                                                       |
| \_\_delitem\_\_                                                                   | Index and slice deletion     | del X[key], del X[i:j]                                                                                                                  |
| \_\_len\_\_                                                                       | Length                       | len(X), truth test if no \_\_bool\_\_                                                                                                   |
| \_\_bool\_\_                                                                      | Boolean tests                | bool(X), truth test (named \_\_nonzero\_\_ in 2.X)                                                                                      |
| \_\_lt\_\_,  \_\_gt\_\_, <br> \_\_le\_\_, \_\_ge\_\_, <br> \_\_eq\_\_, \_\_ne\_\_ | Comparisons                  | X < Y, X > Y, X <= Y, X>= Y, X==Y, X != Y (or else \_\_cmp\_\_ in 2.X only)                                                             |
| \_\_radd\_\_                                                                      | Right-side operators         | Other + X                                                                                                                               |
| \_\_iadd\_\_                                                                      | In-place augmented operators | X += Y (or else \_\_add\_\_)                                                                                                            |
| \_\_iter\_\_, \_\_next\_\_                                                        | Iteration contexts           | `I=iter(x),next(I);for`loops, `in` if no \_\_contains\_\_, `all` comprehensions, `map(F,X)`, others (\_\_next\_\_ is named next in 2.x) |
| \_\_contains\_\_                                                                  | Membership test              | item in X (any iterable)                                                                                                                |
| \_\_index\_\_                                                                     | Integer value                | hex(X),bin(X),oct(X),O[X],O[X:] (replace 2.X \_\_oct\_\_, \_\_hex\_\_)                                                                  |
| \_\_enter\_\_, \_\_exit\_\_                                                       | Context manager              | with obj as var:                                                                                                                        |
| \_\_get\_\_, \_\_set\_\_                                                          | Descriptor attributes        | X.attr, X.attr=value, del X.attr                                                                                                        |
| \_\_new\_\_                                                                       | Creation                     | Object creation, before \_\_init\_\_                                                                                                        |


## Indexing and slicing: __getitem__ and __setitem__

In [ ]:
class Indexer:
    data = list(range(0, 100))
    def __getitem__(self, index):
        if isinstance(index, int):
            print('indexing', index)
        else:
            print('slicing', index.start, index.stop, index.step)
    def __setitem__(self, index, value):
        self.data[index] = value
            
x = Indexer()
x[99]
x[1:99:2]
x[90] = 2
x[90]

## Index iteration: __getitem__

In [ ]:
class StepperIndexer:
    def __getitem__(self, i):
        return self.data[i]
    
x = StepperIndexer()
x.data = 'Spam'
for item in x:
    print(item, end=' ')

## Iterable objects: __iter__ and __next__

Although the `__getitem__` technique works, it's really just a fallback for iteration. All iteration contexts will try the `__iter__` method first, before trying `__getitem__`.

### User-Defined Iterables

In [ ]:
class Squares:
    def __init__(self, start, stop):
        self.value = start - 1
        self.stop = stop
    def __iter__(self):
        return self
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2

for i in Squares(1, 5):
    print(i, end=' ')

### Multiple Iterators on One Object

In [ ]:
class SkipObject:
    def __init__(self, wrapped):
        self.wrapped = wrapped
    def __iter__(self):
        return SkipIterator(self.wrapped)

class SkipIterator:
    def __init__(self, wrapped):
        self.wrapped = wrapped
        self.offset = 0
    def __next__(self):
        if self.offset >= len(self.wrapped):
            raise StopIteration
        else:
            item = self.wrapped[self.offset]
            self.offset += 2
            return item

alpha = 'abcdef'
skipper = SkipObject(alpha)
I = iter(skipper)
print(next(I), next(I), next(I))

for x in skipper:
    for y in skipper:
        print(x + y, end = ' ')

### Coding alternative: __iter__ plus yield

In [ ]:
class Squares:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        for value in range(self.start, self.stop + 1):
            yield value ** 2

## Memberships: \_\_contains\_\_, \_\_iter\_\_, and \_\_getitem\_\_

In the iterations domain, classes can implement the `in` membership operator as an iteration, using either the `__iter__` or `__getitem__` methods. To support more specific membership, though, classes may code a `__contains__` method — when present, this method is preferred over `__iter__`, which is preferred over `__getitem__`. The `__contains__` method should define membership as applying to keys for a _mapping_ (and can use quick lookups), and as a search for _sequences_.


In [ ]:
class Iters:
    def __init__(self, value):
        self.data = value

    def __getitem__(self, i):           # Fallback for iteration
        return self.data[i]             # Also for index, slice

    def __iter__(self):                 # Preferred for iteration
        for x in self.data:             # Allows multiple active iterators
            yield x                     # no __next__ to alias to next

    def __contains__(self, x):          # Preferred for 'in'
        return x in self.data

## Attribute access: \_\_getattr\_\_ and \_\_setattr\_\_

### Attribute reference

The \_\_getattr\_\_ method intercepts attribute references. It's called with  attribute references. It's called with the attribute name as a string whenever you try to qualify an instance with an _undefined_ (nonexistent) attribute name. It is not called if Python can find the attribute using its inheritance tree search procedure.

Because of its behavior, \_\_getattr\_\ is useful as a hook for responding to attribute requests in a generic fashion. It's commonly used to delegate calls to embedded (or wrapped) objects from a proxy controller object. This method can also be used to adapt classes to an interface, or add _accessors_ for data attributes after the fact—logic in a method that validates or computes an attribute after it's already being used with simple dot notation.

In [ ]:
class Empty:
    def __getattr__(self, attrname):
        if attrname == 'age':
            return 40
        else
            raise AttributeError(attrname)

### Attribute assignment and deletion

The \_\_setattr\_\_ intercepts **all** attribute assignments. If this method is defined or inherited, `self.attr = value` become `self.__setattr('attr', value)`. Like `__getattr__`, this allows class to catch attribute changes, and validate or transform as desired.

If the method is used, avoid loops by coding instance attribute assignments as assignments to attribute dictionary.


In [ ]:
class AccessControl:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value + 10
        else:
            raise AttributeError(attr + ' not allowed')